In [ ]:
# python notebook to study NuWro fake data distributions
# need to do manual ppfx correction weights

In [ ]:
import sys

sys.path.insert(0, 'backend_functions')

import selection_functions as sf

import importlib

import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb

import awkward
import matplotlib.pyplot as plt
import pandas as pd

import ROOT
from ROOT import TH1F, TH2F, TDirectory, TH1D

import top 
from top import *

importlib.reload(sf)
from selection_functions import *

In [ ]:
import json

In [ ]:
fold = "nuselection"
tree = "NeutrinoSelectionFilter"

In [ ]:
ISRUN3 = False

In [ ]:
nuwro = False

In [ ]:
fold = "nuselection"
tree = "NeutrinoSelectionFilter"

if nuwro: 
    path = '/uboone/data/users/kmiller/uBNuMI_CCNp/ntuples/run1/cv/'

else: 
    if not ISRUN3:
        path = '/uboone/data/users/kmiller/uBNuMI_CCNp/ntuples/run1/cv_slimmed/qualcuts/'
    else: 
        path = '/uboone/data/users/kmiller/uBNuMI_CCNp/ntuples/run3b/cv_slimmed/qualcuts/'


In [ ]:
if nuwro: 
    overlay = uproot.open(path+"neutrinoselection_filt_run1_overlay_nuwro.root")[fold][tree]
    nue = uproot.open(path+"neutrinoselection_filt_run1_overlay_nuwro_intrinsic.root")[fold][tree]
  
else: 
    if not ISRUN3: 
        overlay = uproot.open(path+"neutrinoselection_filt_run1_overlay_v7.root")[fold][tree]
        nue = uproot.open(path+"neutrinoselection_filt_run1_overlay_intrinsic_v7.root")[fold][tree]
    else: 
        overlay = uproot.open(path+"neutrinoselection_filt_run3b_overlay_v7.root")[fold][tree]
        nue = uproot.open(path+"neutrinoselection_filt_run3b_overlay_intrinsic_v7.root")[fold][tree]
    
uproot_v = [overlay,nue]

In [ ]:

variables = [
    "selected", "nu_pdg", 
    "trk_score_v", 
    "shr_tkfit_dedx_Y",
    "ccnc", 
    "n_tracks_contained", 
    "NeutrinoEnergy2",
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "true_nu_vtx_x", "true_nu_vtx_y" , "true_nu_vtx_z", 
    "nproton", "nu_e", "n_showers_contained",
    "shr_score", 
    "trk_energy", "tksh_distance", "tksh_angle",
    "npi0", 
    "shr_energy_tot_cali", "shr_energy_cali", 
    "nslice",
    "contained_fraction",
    "npion", 
    "shrmoliereavg", 
    "shrsubclusters0","shrsubclusters1","shrsubclusters2", 
    "shr_tkfit_npointsvalid","shr_tkfit_npoints", "true_e_visible", "trkshrhitdist2", 
    "weightSplineTimesTune", "weightTune", "true_nu_px", "true_nu_py", "true_nu_pz", 
    "elec_e", "proton_e", "mc_px", "mc_py", "mc_pz", "elec_px", "elec_py", "elec_pz", 
    "swtrig_pre", "ppfx_cv", "mc_pdg",  'interaction', 'run', "opening_angle"]

In [ ]:
overlay = overlay.pandas.df(variables, flatten=False)

In [ ]:
nue = nue.pandas.df(variables, flatten=False)

In [ ]:
df_v = [overlay,nue]

In [ ]:
for i,df in enumerate(df_v):
    up = uproot_v[i]
    trk_llr_pid_v = up.array('trk_llr_pid_score_v')
    trk_id = up.array('trk_id')-1 # I think we need this -1 to get the right result
    trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
    df['trkpid'] = trk_llr_pid_v_sel
    df['subcluster'] = df['shrsubclusters0'] + df['shrsubclusters1'] + df['shrsubclusters2']

    df['NeutrinoEnergy2_GeV'] = df['NeutrinoEnergy2']/1000


In [ ]:
overlay = overlay.query('swtrig_pre==1')
nue = nue.query('swtrig_pre==1')

In [ ]:
mc_df = [overlay, nue]

In [ ]:
for i,df in enumerate(mc_df):
    
    # is signal bool 
    df['is_signal'] = np.where((df.swtrig_pre == 1) 
                             & (df.nu_pdg==12) & (df.ccnc==0) & (df.nproton>0) & (df.npion==0) & (df.npi0==0)
                             & (10 <= df.true_nu_vtx_x) & (df.true_nu_vtx_x <= 246)
                             & (-106 <= df.true_nu_vtx_y) & (df.true_nu_vtx_y <= 106)
                             & (10 <= df.true_nu_vtx_z) & (df.true_nu_vtx_z <= 1026), True, False)
    
    # Add truth level theta & phi angles (detector & beam coordinates)
    df = addAngles(df)
    
    # add binned ppfx weights
    if nuwro: 
        df = offline_flux_weights(df, ISRUN3)
    

In [ ]:
print(len(nue.query('is_signal==True'))==len(nue.query(signal)))
print(len(nue.query('is_signal==False'))==len(nue.query(not_signal)))

## Weights

In [ ]:
# clean bad weights & values 

for i,df in enumerate(mc_df):
     
    df.loc[ df['weightSplineTimesTune'] <= 0, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] == np.inf, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] > 100, 'weightSplineTimesTune' ] = 1.
    df.loc[ np.isnan(df['weightSplineTimesTune']) == True, 'weightSplineTimesTune' ] = 1.
    
    df.loc[ df['weightTune'] <= 0, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] == np.inf, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] > 100, 'weightTune' ] = 1.
    df.loc[ np.isnan(df['weightTune']) == True, 'weightTune' ] = 1.  

     

In [ ]:
# POT normalization - scale to data 

if ISRUN3: 
    data_pot = 5.0E20
else: 
    data_pot = 2.0E20

if nuwro==True: 
    overlay['pot_scale'] = data_pot/6.64422e+20
    nue['pot_scale'] = data_pot/1.68375e+22

else: 
    overlay = pot_scale(overlay, 'overlay', ISRUN3)
    nue = pot_scale(nue, 'intrinsic', ISRUN3)
    
    #overlay['pot_scale'] = data_pot/2.33652e+21
    #nue['pot_scale'] = data_pot/2.37838e+22


In [ ]:
if nuwro: 

    overlay['totweight_data'] = overlay['pot_scale']*overlay['ppfx_cv']
    nue['totweight_data'] = nue['pot_scale']*nue['ppfx_cv']

else: 
    
    overlay['totweight_data'] = overlay['pot_scale']*overlay['ppfx_cv']*overlay['weightSplineTimesTune']
    nue['totweight_data'] = nue['pot_scale']*nue['ppfx_cv']*nue['weightSplineTimesTune']

## Categories

In [ ]:
print("# of nueCC in AV in overlay sample = "+str(len(overlay.query(nueCC_query))))
len1 = len(overlay)
    
idx = overlay.query(nueCC_query).index
overlay.drop(idx, inplace=True)
len2 = len(overlay) 
print("# of nueCC in AV dropped in overlay = "+str(len1-len2))
    
# then add in nue_intrinsic 
overlay = pd.concat([overlay,nue], ignore_index=True)

# from here on out everything else should be the same. 

In [ ]:
# separate by in/out FV & cosmic
infv = overlay.query(in_fv_query)
outfv = overlay.query(out_fv_query)

In [ ]:
# check that everything is accounted for 
print(len(overlay)==len(infv)+len(outfv))

In [ ]:
tot_signal_weighted = np.nansum(infv.query('is_signal==True')['pot_scale'])
print('total signal events = '+ str(tot_signal_weighted))


In [ ]:
datasets = {
    'infv' : infv, 
    'outfv' : outfv
}

## BDT selection

In [ ]:
datasets_bdt = {}

bdt_score_cut = parameters(ISRUN3)['bdt_score_cut']
#selected_query = BDT_LOOSE_CUTS +' and BDT_score>='+str(bdt_score_cut)
selected_query = BDT_LOOSE_CUTS +' and BDT_score>=0.4 and BDT_score<=0.5'  #str(bdt_score_cut)
    
#print("BDT SCORE THRESHOLD = "+str(bdt_score_cut))

print(selected_query)

In [ ]:
# load bdt model 
split = 1
bdt_model = xgb.Booster({'nthread': 4})
bdt_model.load_model(parameters(ISRUN3)['bdt_model'])

for i in range(len(datasets)): 

    df = list(datasets.values())[i].copy()
    df = df.query(BDT_LOOSE_CUTS)

    # clean datasets 
    for column in training_parameters:
        df.loc[(df[column] < -1.0e37) | (df[column] > 1.0e37), column] = np.nan

    # create testing dmatrix 
    df_test = xgb.DMatrix(data=df[training_parameters])

    # apply the bdt selection
    preds = bdt_model.predict(df_test)

    # add columns for plotting 
    df['BDT_score'] = preds

    datasets_bdt[list(datasets.keys())[i]] = df.query(selected_query)

## Selected distributions

In [ ]:
xvar = 'tksh_angle'
true_var = 'opening_angle'

bins = [-1, -0.5, np.cos(100 * np.pi/180), np.cos(80 * np.pi/180), 0.5, 1]
x_ticks = [-1, -0.5, -0.174, 0.174, 0.5, 1]

xlow = bins[0]
xhigh = bins[-1]

x_label = "cos $\\theta_{ep}$"

In [ ]:
if xvar=='tksh_angle': 
    bincenters = 0.5*(np.array(x_ticks)[1:]+np.array(x_ticks)[:-1])

else: 
    bincenters = 0.5*(np.array(x_ticks+[xhigh])[1:]+np.array(x_ticks+[xhigh])[:-1])

In [ ]:
leg = {
        'outfv' : labels['outfv'][0]+': ' + str( round(sum(datasets_bdt['outfv'].totweight_data), 1)) , 
        'numu_NC_Npi0' : labels['numu_NC_Npi0'][0]+': '+ str( round(sum(datasets_bdt['infv'].query(numu_NC_Npi0).totweight_data), 1)), 
        'numu_CC_Npi0' : labels['numu_CC_Npi0'][0]+': '+str( round(sum(datasets_bdt['infv'].query(numu_CC_Npi0).totweight_data), 1)), 
        'numu_NC_0pi0' : labels['numu_NC_0pi0'][0]+': '+str( round(sum(datasets_bdt['infv'].query(numu_NC_0pi0).totweight_data), 1)), 
        'numu_CC_0pi0' : labels['numu_CC_0pi0'][0]+': '+str( round(sum(datasets_bdt['infv'].query(numu_CC_0pi0).totweight_data), 1)), 
        'nue_NC' : labels['nue_NC'][0]+': '+str( round(sum(datasets_bdt['infv'].query(nue_NC).totweight_data), 1)), 
        'nue_CCother' : labels['nue_CCother'][0]+': '+str( round(sum(datasets_bdt['infv'].query(nue_CCother).totweight_data), 1)),
        'nuebar_1eNp' : labels['nuebar_1eNp'][0]+': '+str( round(sum(datasets_bdt['infv'].query(nuebar_1eNp).totweight_data), 1)), 
        'signal' : labels['signal'][0]+': '+str( round(sum(datasets_bdt['infv'].query(signal).totweight_data), 1))
    }

In [ ]:
fig = plt.figure(figsize=(8, 5))

n, b, p = plt.hist([datasets_bdt['outfv'][xvar], 
                       datasets_bdt['infv'].query(numu_NC_Npi0)[xvar],
                       datasets_bdt['infv'].query(numu_CC_Npi0)[xvar],
                       datasets_bdt['infv'].query(numu_NC_0pi0)[xvar],
                       datasets_bdt['infv'].query(numu_CC_0pi0)[xvar],
                       datasets_bdt['infv'].query(nue_NC)[xvar],
                       datasets_bdt['infv'].query(nue_CCother)[xvar],
                       datasets_bdt['infv'].query(nuebar_1eNp)[xvar], 
                       datasets_bdt['infv'].query(signal)[xvar]],
            bins, histtype='bar', range=[xlow, xhigh], stacked=True, 
            color=[labels['outfv'][1], 
                       labels['numu_NC_Npi0'][1], 
                       labels['numu_CC_Npi0'][1], 
                       labels['numu_NC_0pi0'][1], 
                       labels['numu_CC_0pi0'][1], 
                       labels['nue_NC'][1], 
                       labels['nue_CCother'][1],
                       labels['nuebar_1eNp'][1], 
                       labels['signal'][1]], 
            label=[leg['outfv'], 
                   leg['numu_NC_Npi0'], 
                   leg['numu_CC_Npi0'], 
                   leg['numu_NC_0pi0'], 
                   leg['numu_CC_0pi0'], 
                   leg['nue_NC'], 
                   leg['nue_CCother'], 
                   leg['nuebar_1eNp'], 
                   leg['signal']
                  ],
            weights=[datasets_bdt['outfv'].totweight_data, 
                     datasets_bdt['infv'].query(numu_NC_Npi0).totweight_data, 
                     datasets_bdt['infv'].query(numu_CC_Npi0).totweight_data,
                     datasets_bdt['infv'].query(numu_NC_0pi0).totweight_data,
                     datasets_bdt['infv'].query(numu_CC_0pi0).totweight_data,
                     datasets_bdt['infv'].query(nue_NC).totweight_data,
                     datasets_bdt['infv'].query(nue_CCother).totweight_data,
                     datasets_bdt['infv'].query(nuebar_1eNp).totweight_data,
                     datasets_bdt['infv'].query(signal).totweight_data 
                     ])

# simulation outline 
tot = list([0, n[-1][0]])+list(n[-1])+[0]
b_step = list([b[0]])+list(b)+list([b[-1]])
plt.step(b_step, tot, color='saddlebrown', linewidth=2)

# plot format stuff
plt.legend(loc='best', prop={"size":10}, ncol=3, frameon=False)


    
#plt.xlabel("Reconstructed "+x_label, fontsize=15)
plt.xlabel(x_label, fontsize=15)

#plt.xticks(x_ticks, fontsize=14)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.xlim(xlow, xhigh)
plt.ylim(0, 10)

if not ISRUN3: 
    plt.title("FHC Run 1 - MC Only", fontsize=16)
    plt.ylabel('$\\nu$ / $2\\times10^{20}$ POT', fontsize=15)
  
else: 
    plt.title("RHC Run 3 - MC Only", fontsize=16)
    plt.ylabel('$\\nu$ / $5\\times10^{20}$ POT', fontsize=15)

#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/fhc/fakedata/nuwro_evt_rate_"+xvar+".pdf", 
#            transparent=True, bbox_inches='tight')

plt.show()
          

In [ ]:
n[-1]

In [ ]:
selected_events = pd.concat([datasets_bdt['outfv'], datasets_bdt['infv']])

In [ ]:
# ppfx correction effect 
fig = plt.figure(figsize=(8, 5))

plt.hist(selected_events[xvar], bins, range=[xlow, xhigh],
            weights=selected_events.pot_scale, linestyle=":", linewidth=2, histtype='step', 
                  label='Unweighted')

total = plt.hist(selected_events[xvar], bins, range=[xlow, xhigh],
            weights=selected_events.totweight_data, linestyle=":", linewidth=2, histtype='step', 
                  label='PPFX-corrected')[0]


# plot format stuff
plt.legend(loc='best', prop={"size":14}, ncol=1, frameon=False)

plt.ylabel('$\\nu$ / $2\\times10^{20}$ POT', fontsize=15)
    
plt.xlabel("Reco "+x_label, fontsize=15)
plt.xticks(x_ticks, fontsize=14)

plt.yticks(fontsize=14)

plt.xlim(xlow, xhigh)
plt.title("Selected Event Rate", fontsize=16)

plt.show()

total

In [ ]:
# signal 

# ppfx correction effect 
fig = plt.figure(figsize=(8, 5))

selected_signal = plt.hist(selected_events.query("is_signal==True")[xvar], bins, range=[xlow, xhigh],
            weights=selected_events.query("is_signal==True")['totweight_data'], linestyle=":", linewidth=2, histtype='step', 
                  label='signal')[0]


# plot format stuff
plt.legend(loc='best', prop={"size":14}, ncol=1, frameon=False)

plt.ylabel('$\\nu$ / $2\\times10^{20}$ POT', fontsize=15)
    
plt.xlabel("Reco "+x_label, fontsize=15)
plt.xticks(x_ticks, fontsize=14)

plt.yticks(fontsize=14)

plt.xlim(xlow, xhigh)
plt.ylim(0,10)

plt.show()

selected_signal

In [ ]:
# background

# signal 

fig = plt.figure(figsize=(8, 5))

selected_bkgd = plt.hist(selected_events.query("is_signal==False")[xvar], bins, range=[xlow, xhigh],
            weights=selected_events.query("is_signal==False")['totweight_data'], linestyle=":", linewidth=2, histtype='step', 
                  label='background')[0]


# plot format stuff
plt.legend(loc='best', prop={"size":14}, ncol=1, frameon=False)

plt.ylabel('$\\nu$ / $2\\times10^{20}$ POT', fontsize=15)
    
plt.xlabel("Reco "+x_label, fontsize=15)
plt.xticks(x_ticks, fontsize=14)

plt.yticks(fontsize=14)

plt.xlim(xlow, xhigh)
plt.ylim(0,10)

plt.show()

selected_bkgd

In [ ]:
 selected_signal

In [ ]:
total

## GENIE v. NuWro comparisons - near sideband (0.4-0.5)

In [ ]:
nuwro_sideband = { 
    "total" : [5.79914398, 3.18000362, 5.10234383, 4.59461656, 6.10006114], # ext not included
    "signal" : [1.78678169, 1.65326607, 2.2762863 , 1.6612245 , 1.18996308], 
    "nu_bkgd" : [4.01236229, 1.52673754, 2.82605753, 2.93339206, 4.91009806]
} 

In [ ]:
genie_sideband = { 
    "total" :  [5.31846251, 2.81177868, 5.34921353, 3.53546428, 6.01169227], # ext not included
    "signal" : [2.12954904, 1.62010666, 2.53347167, 1.58859615, 1.64588623], 
    "nu_bkgd" : [3.18891347, 1.19167202, 2.81574186, 1.94686813, 4.36580604]
    #"total_bkgd" : 
} 


In [ ]:
fig = plt.figure(figsize=(8, 5))

# genie
n_genie, b_genie, p_genie = plt.hist(bincenters, bins, histtype='step', range=[xlow,xhigh], 
         weights=genie_sideband['signal'], 
         linestyle=":", linewidth=2, label='GENIE v3')

# nuwro
n_nuwro, b_nuwro, p_nuwro = plt.hist(bincenters, bins, histtype='step', range=[xlow,xhigh], 
         weights=[a-b for a,b in zip(nuwro_sideband['total'],genie_sideband['nu_bkgd'])], 
         linestyle=":", linewidth=2, label='NuWro')


# plot format stuff
plt.legend(loc='best', prop={"size":14}, ncol=1, frameon=False)

plt.ylabel('$\\nu$ / $2\\times10^{20}$ POT', fontsize=15)
    
plt.xlabel("Reconstructed "+x_label, fontsize=15)
plt.xticks(x_ticks, fontsize=14)

plt.yticks(fontsize=14)

plt.xlim(xlow, xhigh)
plt.ylim(0, 7)

plt.title("GENIE Background-Subtracted Evt Rate", fontsize=16)

plt.show()

In [ ]:
bins

In [ ]:
importlib.reload(uncertainty_functions)
from uncertainty_functions import * 

In [ ]:
# compute the covariance

# ncv_nu = background subtracted neutrino event rate 
# ncv_total = background subtracted event rate (including EXT - needed for fractional covariance)

cov_dict = calcCov(xvar, bins, genie_sideband['signal'], genie_sideband['signal'], 
        [[a-b for a,b in zip(nuwro_sideband['total'],genie_sideband['nu_bkgd'])]], 
        plot=True, save=False, axis_label=None, pot=None, isrun3=False, xticks=x_ticks, xhigh=xhigh)

In [ ]:
cov_dict['fractional_uncertainty']

In [ ]:
near_dict = {
    "genie_evt_rate" : genie_sideband['signal'], # background subtracted evt rate 
    "nuwro_evt_rate" : [a-b for a,b in zip(nuwro_sideband['total'],genie_sideband['nu_bkgd'])], 
    "cov" : cov_dict['cov'], 
    "frac_cov" : cov_dict['frac_cov'], 
    "genie_nu_bkgd" : genie_sideband['nu_bkgd'], 
    "nuwro_nu_bkgd" : nuwro_sideband['nu_bkgd']
}

In [ ]:
# save to file: 
# background subtracted event rates used 
# absolute covariance 
# fractional covariance
# for reweights: genie nu background, nuwro nu background 

with open('unfolding/NearSideBand'+xvar+'_NewBinning.json', "w") as f:
        json.dump(near_dict, f)

## GENIE v. NuWro comparisons - far sideband

In [ ]:
nuwro_bkgd_sideband = {
    "full_bdt_score" : [ 8.04361205, 27.17178132, 28.44883151, 23.86514134, 24.77616913,
       23.72546924, 28.95150452, 25.20916465,  4.84824916,  0.        ], 
    "electron_energy" : [41.71487162, 15.04154548,  7.53057903,  4.90490485,  6.98261312,
        3.01947008], 
    "visible_energy" : [42.00215307, 12.9403889 ,  7.1606288 ,  5.61335851,  4.32586293,
        4.12098612,  2.18924797], 
    "opening_angle" : [16.18735688,  8.79454785, 11.97147006, 13.32774862, 11.56595642,
       17.63867432], 
    "ntracks" : [51.34162897, 19.16072888,  8.98339631]
    
}

genie_fhc_bkgd_sideband = {
    "full_bdt_score" : [11.5080753 , 28.81665718, 30.24147479, 27.08685742, 23.02661127,
       28.51794976, 31.12420486, 26.32135832,  4.88495939,  0.        ], 
    "electron_energy" : [46.44829683, 16.07459317,  8.09514408,  5.64018824,  5.77522167,
        3.11146339],
    "visible_energy" : [46.32888965, 13.04610779,  8.38904512,  5.12928039,  5.65242695,
        3.55603469,  2.49263497], 
    "opening_angle" : [16.72473781, 10.90117788, 12.2597639 , 14.10559867, 14.21070621,
       17.94300492], 
    "ntracks" : [52.09513431, 23.02299444, 11.02686064]
}



In [ ]:
# normalized to 5E20 POT 
genie_rhc_bkgd_sideband = {
    "electron_energy" : [101.01842554,  36.19685743,  17.59088361,  12.40675679,
        13.87249858,   9.64100869],
    "visible_energy" : [108.91075175,  25.12923249,  16.45246847,   8.92139437,
        13.03065202,   8.92899277,   7.73096787],
    "opening_angle" : [42.38227477, 19.29432241, 27.81925245, 30.61516307, 30.76603424,
       41.92771332],
    "ntracks" : [113.94541603,  52.19761961,  26.66172461]
}

In [ ]:
xvar = 'shr_energy_cali'
true_var = 'elec_e'
bins = [0.02, 0.22, 0.42, 0.62, 0.82 , 1.22,  7]

xlow = bins[0]
xhigh = 2.5

x_label = "Electron Energy [GeV]"

x_ticks = [0.02, 0.22, 0.42, 0.62, 0.82 , 1.22]


In [ ]:
bincenters = 0.5*(np.array(bins)[1:]+np.array(bins)[:-1])

In [ ]:
# comparisons

fig = plt.figure(figsize=(8, 5))

# genie fhc
plt.hist(bincenters, bins, histtype='step', range=[xlow,xhigh], 
        weights=genie_fhc_bkgd_sideband['electron_energy'], linestyle=":", linewidth=2, 
                  label='GENIE FHC', color='limegreen')

# genie rhc
plt.hist(bincenters, bins, histtype='step', range=[xlow,xhigh], 
        weights=[a*(2E20/5E20) for a in genie_rhc_bkgd_sideband['electron_energy']], linestyle=":", linewidth=2, 
                  label='GENIE RHC', color='red')

# nuwro

#plt.hist(bincenters, bins, histtype='step', 
#         weights=nuwro_bkgd_sideband['ntracks'], range=[xlow,xhigh], 
#         linestyle=":", linewidth=2, 
#                  label='NuWro')

# plot format stuff
plt.legend(loc='best', prop={"size":14}, ncol=2, frameon=False)

plt.ylabel('$\\nu$ / $2\\times10^{20}$ POT', fontsize=15)
    
plt.xlabel("Reconstructed "+x_label, fontsize=15)
#plt.xlabel("BDT score", fontsize=15)
plt.xticks(x_ticks, fontsize=14)

plt.yticks(fontsize=14)

plt.xlim(xlow,xhigh)
plt.ylim(0, 50)
#plt.title("MC Background Distribution", fontsize=16)


plt.show()

## save to file

In [ ]:
f_fhc.close()

In [ ]:
fhc_dict.keys()

In [ ]:
fhc_dict['nuwro_full_sumw2'] = nuwro_full_evt_rate_sumw2

In [ ]:
if xvar=='shr_energy_cali': 

    with open('unfolding/variations/FHCVariations_'+xvar+'_July15.json', "w") as f_fhc:
        json.dump(fhc_dict, f_fhc)
        
else: 
    with open('unfolding/variations/FHCVariations_'+xvar+'_July19.json', "w") as f_fhc:
        json.dump(fhc_dict, f_fhc)

## replace distributions

In [ ]:
# reco distribution (MC or data)
hreco = TH1D("hmeas", "Selected NuWro Signal vs. Reco "+x_label+" ("+xvar+")", len(bins)-1, np.array(bins))

for i in range(len(bincenters)): 
    hreco.Fill(bincenters[i], fhc_dict['evt_rate_nuwro_ppfx'][i])


In [ ]:
f = ROOT.TFile.Open('/uboone/data/users/kmiller/uBNuMI_CCNp/unfolding/fakedata/input/WSVD_'+xvar+'_FHC_NuWroToGENIE_ppfx.root', "UPDATE")


In [ ]:
f.cd()
hreco.Write()

f.Close()

In [ ]:
# generated signal 

htrue_signal = TH1F("htrue_signal", "Generated NuWro MC Signal vs. True "+x_label+" ("+true_var+")", len(bins)-1, np.array(bins))

for i in range(len(bincenters)): 
    htrue_signal.Fill(bincenters[i], fhc_dict['nuwro_generated_signal_ppfx'][i])
    print(fhc_dict['nuwro_generated_signal_ppfx'][i])

In [ ]:
f = ROOT.TFile.Open('/uboone/data/users/kmiller/uBNuMI_CCNp/unfolding/fakedata/input/WSVD_'+xvar+'_FHC_NuWroToNuWro_ppfx.root', "UPDATE")


In [ ]:
f.cd()
#hreco.Write()
htrue_signal.Write()

f.Close()